save color gt_instance grounded_sam maskrcnn

In [1]:
import os
import sys

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"

module_path = os.path.abspath(os.path.join("../.."))
if module_path not in sys.path:
    sys.path.append(module_path)


In [2]:
import json

import matplotlib.pyplot as plt
import numpy as np
import open3d as o3d
import torch
from PIL import Image

from core.integrate import FeatureFusionScalableTSDFVolume, PanopticFusionScalableTSDFVolume
from core.dataset import ScanNet
from core.labeler import CLIPTextQuerier, KMeansLabeler


In [3]:
o3d._build_config["ENABLE_HEADLESS_RENDERING"]


True

### Save color

In [4]:
device = "cuda:0"
scenes = ["scene0000_00", "scene0645_01", "scene0643_00", "scene0488_01"]


In [5]:
save_dir = "../05_panoptic_fusion/"
sub_save_dir = save_dir + "01_color/"


In [6]:
sample_data = o3d.data.DemoCustomVisualization()


In [ ]:
for scene in scenes:
    json_pth = os.path.abspath("../../config/views/scannet_" + scene + ".json")
    with open(json_pth, "r") as f:
        views = json.load(f)["views"]

    load_dir = "/scratch/quanta/Experiments/feature-instance-fusion/scannet_" + scene + "/"

    tsdf_volume = FeatureFusionScalableTSDFVolume(
        voxel_size=0.015,
        sdf_trunc=0.075,
        margin=0.08,
        device=device,
    )
    tsdf_volume.load(load_dir + "tsdf/tsdf_volume.pt")
    tsdf_volume.reset_feature(dim=3, include_var=False, dtype=torch.float32)
    tsdf_volume.load_feats(load_dir + "tsdf_feature_color/feats.pt")
    verts = np.load(load_dir + "tsdf/verts.npy")
    faces = np.load(load_dir + "tsdf/faces.npy")
    mesh = o3d.geometry.TriangleMesh(
        vertices=o3d.utility.Vector3dVector(verts),
        triangles=o3d.utility.Vector3iVector(faces),
    )

    color = tsdf_volume.extract_feat_on_grid(verts=verts)[0]

    mesh.vertex_colors = o3d.utility.Vector3dVector(color)

    o3d.io.write_triangle_mesh(scene[-7:] + '_color.ply', mesh)
    # break

    # for i, view in enumerate(views):
    #     tmp_save_dir = sub_save_dir + scene + "/"
    #     os.makedirs(tmp_save_dir, exist_ok=True)
    #     tmp_save_name = tmp_save_dir + "{:02d}_".format(i) + view["tag"] + ".png"

    #     vis = o3d.visualization.Visualizer()
    #     vis.create_window()
    #     vis.add_geometry(mesh)
    #     vis.set_view_status(json.dumps(view["view"]))
    #     buffer = vis.capture_screen_float_buffer(do_render=True)
    #     image = Image.fromarray((np.asarray(buffer) * 255).astype(np.uint8))
    #     # display(image)
    #     image.save(tmp_save_name)


In [ ]:
draw = o3d.visualization.EV.draw

draw([mesh])


### panoptic fusion

In [9]:
extractors = ["gt_instance", "grounded_sam", "mask_rcnn"]
# extractors = ["grounded_sam"]
# extractors = ["gt_instance"]


In [11]:
for i, ext in enumerate(extractors):
    sub_save_dir = save_dir + "{:02d}_".format(2 + i) + ext + "/"
    for scene in scenes:
        json_pth = os.path.abspath("../../config/views/scannet_" + scene + ".json")
        with open(json_pth, "r") as f:
            views = json.load(f)["views"]

        load_dir = "/scratch/quanta/Experiments/feature-instance-fusion/scannet_" + scene + "/"

        tsdf_volume = PanopticFusionScalableTSDFVolume(
            voxel_size=0.015,
            sdf_trunc=0.075,
            margin=0.08,
            device=device,
        )
        tsdf_volume.load(load_dir + "tsdf/tsdf_volume.pt")
        tsdf_volume.reset_instance()
        tsdf_volume.load_instance(load_dir + "tsdf_panoptic_" + ext + "/panoptic_labels.pt")

        print(tsdf_volume._instance_label_num)

        # verts = np.load(load_dir + "tsdf/verts.npy")
        # faces = np.load(load_dir + "tsdf/faces.npy")

        # color_set = np.random.random(size=(tsdf_volume._instance_label_num, 3))

        # labels = tsdf_volume.extract_label_on_grid(verts)[0]
        # print(np.unique(labels).shape[0])
        # color = color_set[labels]
        # mesh = o3d.geometry.TriangleMesh(
        #     vertices=o3d.utility.Vector3dVector(verts),
        #     triangles=o3d.utility.Vector3iVector(faces),
        # )
        # mesh.vertex_colors = o3d.utility.Vector3dVector(color)

        # break
        # for i, view in enumerate(views):
        #     tmp_save_dir = sub_save_dir + scene + "/"
        #     # print(tmp_save_dir)
        #     os.makedirs(tmp_save_dir, exist_ok=True)
        #     tmp_save_name = tmp_save_dir + "{:02d}_".format(i) + view["tag"] + ".png"

        #     vis = o3d.visualization.Visualizer()
        #     vis.create_window()
        #     vis.add_geometry(mesh)
        #     vis.set_view_status(json.dumps(view["view"]))
        #     buffer = vis.capture_screen_float_buffer(do_render=True)
        #     image = Image.fromarray((np.asarray(buffer) * 255).astype(np.uint8))
        #     # display(image)
        #     image.save(tmp_save_name)

# draw = o3d.visualization.EV.draw

# # draw([mesh])

# o3d.io.write_triangle_mesh('0000_00_panoptic_gsam.ply', mesh)


3056
2831
550
125
13552
12797
3571
3361
9545
2673
1726
167


### graph connect

In [ ]:
sub_save_dir = save_dir + "05_graph_connect/"
for scene in scenes:
    json_pth = os.path.abspath("../../config/views/scannet_" + scene + ".json")
    with open(json_pth, "r") as f:
        views = json.load(f)["views"]

    load_dir = "/scratch/quanta/Experiments/feature-instance-fusion/scannet_" + scene + "/"

    tsdf_volume = PanopticFusionScalableTSDFVolume(
        voxel_size=0.015,
        sdf_trunc=0.075,
        margin=0.08,
        device=device,
    )
    tsdf_volume.load(load_dir + "tsdf/tsdf_volume.pt")
    tsdf_volume.reset_instance()
    tsdf_volume.load_instance(load_dir + "tsdf_panoptic_gt_instance/panoptic_labels.pt")
    tsdf_volume._instance = torch.load(load_dir + "graph_connect_etx-grounded_sam_kmeans-ext-random_grounded_sam_SVP/merged_labels.pt").to(device)

    verts = np.load(load_dir + "tsdf/verts.npy")
    faces = np.load(load_dir + "tsdf/faces.npy")

    color_set = np.random.random(size=(1025, 3))

    labels = tsdf_volume.extract_label_on_grid(verts)[0]
    color = color_set[labels]
    mesh = o3d.geometry.TriangleMesh(
        vertices=o3d.utility.Vector3dVector(verts),
        triangles=o3d.utility.Vector3iVector(faces),
    )

    mesh.vertex_colors = o3d.utility.Vector3dVector(color)

    o3d.io.write_triangle_mesh(scene[-7:] + '_graph_connect_gsam.ply', mesh)
    # break

# draw = o3d.visualization.EV.draw

# draw([mesh])

# o3d.io.write_triangle_mesh('0000_00_panoptic_gsam.ply', mesh)
    # for i, view in enumerate(views):
    #     tmp_save_dir = sub_save_dir + scene + "/"
    #     # print(tmp_save_dir)
    #     os.makedirs(tmp_save_dir, exist_ok=True)
    #     tmp_save_name = tmp_save_dir + "{:02d}_".format(i) + view["tag"] + ".png"

    #     vis = o3d.visualization.Visualizer()
    #     vis.create_window()
    #     vis.add_geometry(mesh)
    #     vis.set_view_status(json.dumps(view["view"]))
    #     # vis.get_render_option().load_from_json(sample_data.render_option_path)
    #     buffer = vis.capture_screen_float_buffer(do_render=True)
    #     image = Image.fromarray((np.asarray(buffer) * 255).astype(np.uint8))
    #     # display(image)
    #     image.save(tmp_save_name)


### Guided panoptic fusion using graph connect results

In [ ]:
sub_save_dir = save_dir + "06_guided_panoptic_fusion/"
for scene in scenes:
    json_pth = os.path.abspath("../../config/views/scannet_" + scene + ".json")
    with open(json_pth, "r") as f:
        views = json.load(f)["views"]

    load_dir = "/home/quanta/Experiments/feature-instance-fusion/scannet_" + scene + "/"

    tsdf_volume = PanopticFusionScalableTSDFVolume(
        voxel_size=0.015,
        sdf_trunc=0.075,
        margin=0.08,
        device=device,
    )
    tsdf_volume.load(load_dir + "tsdf/tsdf_volume.pt")
    tsdf_volume.reset_instance()
    tsdf_volume.load_instance(load_dir + "tsdf_guided_panoptic_grounded_sam/panoptic_labels.pt")

    verts = np.load(load_dir + "tsdf/verts.npy")
    faces = np.load(load_dir + "tsdf/faces.npy")

    color_set = np.random.random(size=(1025, 3))

    labels = tsdf_volume.extract_label_on_grid(verts)[0]
    color = color_set[labels]
    mesh = o3d.geometry.TriangleMesh(
        vertices=o3d.utility.Vector3dVector(verts),
        triangles=o3d.utility.Vector3iVector(faces),
    )

    mesh.vertex_colors = o3d.utility.Vector3dVector(color)
    for i, view in enumerate(views):
        tmp_save_dir = sub_save_dir + scene + "/"
        # print(tmp_save_dir)
        os.makedirs(tmp_save_dir, exist_ok=True)
        tmp_save_name = tmp_save_dir + "{:02d}_".format(i) + view["tag"] + ".png"

        vis = o3d.visualization.Visualizer()
        vis.create_window()
        vis.add_geometry(mesh)
        vis.set_view_status(json.dumps(view["view"]))
        # vis.get_render_option().load_from_json(sample_data.render_option_path)
        buffer = vis.capture_screen_float_buffer(do_render=True)
        image = Image.fromarray((np.asarray(buffer) * 255).astype(np.uint8))
        # display(image)
        image.save(tmp_save_name)
